In [ ]:
#!/usr/bin/env python3
"""
天氣預報資料向量化與檢索系統

使用 Pinecone Inference API 進行文本嵌入，並儲存到 Pinecone 進行向量檢索
專注於臺中市天氣資料的處理與查詢
"""

import os
import re
from typing import List, Dict, Any, Optional
from datetime import datetime
import json
import urllib.parse

from pinecone import Pinecone
from tqdm import tqdm
import time


class WeatherVectorizer:
    """天氣資料向量化處理器"""
    
    def __init__(self, pinecone_api_key: str):
        """
        初始化向量化處理器
        
        Args:
            pinecone_api_key: Pinecone API 金鑰
        """
        # 初始化 Pinecone
        self.pc = Pinecone(api_key=pinecone_api_key)
        self.index_name = "weather-txt-index"
        self.index = self.pc.Index(self.index_name)
        
        # 使用 Pinecone 的 Inference API
        self.embedding_model = "llama-text-embed-v2"
        
        print(f"✅ 已初始化 Pinecone 索引: {self.index_name}")
        print(f"✅ 已初始化 Pinecone 嵌入模型: {self.embedding_model}")
    
    def extract_taichung_data(self, file_path: str) -> List[Dict[str, Any]]:
        """
        從天氣預報文件中提取臺中市的資料
        
        Args:
            file_path: 天氣預報文件路徑
            
        Returns:
            包含臺中市各區天氣資料的列表
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # 解析臺中市各區的資料
            districts_data = self._parse_districts_data(content)
            
            print(f"✅ 成功提取臺中市 {len(districts_data)} 筆天氣資料")
            return districts_data
            
        except Exception as e:
            print(f"❌ 提取臺中市資料時發生錯誤: {e}")
            return []
    
    def _parse_districts_data(self, content: str) -> List[Dict[str, Any]]:
        """解析臺中市各區的天氣資料"""
        districts_data = []
        
        # 找到所有包含臺中市區域資料的段落
        # 模式：在 時間 期間，臺中市XX區的天氣狀況如下：
        pattern = r'在 ([\d\-T\+:]+) 至 ([\d\-T\+:]+) 期間，臺中市(\w+)的天氣狀況如下：\s*\n\s*- 平均溫度：([\d.]+)°C\s*\n\s*- 最低溫度：([\d.]+)°C\s*\n\s*- 最高溫度：([\d.]+)°C'
        
        matches = re.findall(pattern, content)
        
        for match in matches:
            start_time, end_time, district, avg_temp, min_temp, max_temp = match
            
            time_range = f"{start_time} 至 {end_time}"
            
            weather_data = {
                "district": district,
                "time_range": time_range,
                "avg_temp": float(avg_temp),
                "min_temp": float(min_temp),
                "max_temp": float(max_temp),
                "text_description": self._create_text_description(
                    district, time_range, avg_temp, min_temp, max_temp
                )
            }
            districts_data.append(weather_data)
        
        return districts_data
    
    def _create_text_description(self, district: str, time_range: str, 
                                avg_temp: str, min_temp: str, max_temp: str) -> str:
        """為天氣資料創建描述性文本"""
        # 解析時間範圍
        time_parts = time_range.split(" 至 ")
        if len(time_parts) == 2:
            start_time = time_parts[0].replace("T", " ").replace("+08:00", "")
            end_time = time_parts[1].replace("T", " ").replace("+08:00", "")
            
            description = (
                f"臺中市{district}在{start_time}至{end_time}期間的天氣預報：" +
                f"平均溫度為{avg_temp}度，最低溫度{min_temp}度，最高溫度{max_temp}度。"
            )
        else:
            description = (
                f"臺中市{district}的天氣預報：" +
                f"平均溫度為{avg_temp}度，最低溫度{min_temp}度，最高溫度{max_temp}度。"
            )
        
        return description
    
    def create_embeddings(self, texts: List[str]) -> List[List[float]]:
        """
        使用 Pinecone Inference API 創建文本嵌入向量
        
        Args:
            texts: 要嵌入的文本列表
            
        Returns:
            嵌入向量列表
        """
        embeddings = []
        
        print("🔄 正在生成文本嵌入向量...")
        
        # 使用 Pinecone Inference API 批次處理
        batch_size = 32  # Pinecone 建議的批次大小
        
        for i in tqdm(range(0, len(texts), batch_size), desc="生成嵌入向量"):
            batch_texts = texts[i:i + batch_size]
            
            try:
                # 使用 Pinecone Inference API
                response = self.pc.inference.embed(
                    model=self.embedding_model,
                    inputs=batch_texts,
                    parameters={"input_type": "passage"}
                )
                
                # 提取嵌入向量
                for embedding_data in response.data:
                    embeddings.append(embedding_data.values)
                
                # 避免 API 限制
                time.sleep(0.1)
                
            except Exception as e:
                print(f"❌ 生成嵌入向量時發生錯誤: {e}")
                # 使用零向量作為備用
                for _ in batch_texts:
                    embeddings.append([0.0] * 1024)  # llama-text-embed-v2 的維度
        
        print(f"✅ 成功生成 {len(embeddings)} 個嵌入向量")
        return embeddings
    
    def _create_safe_vector_id(self, district: str, index: int) -> str:
        """
        創建符合 ASCII 要求的向量 ID
        
        Args:
            district: 區名（可能包含中文）
            index: 索引編號
            
        Returns:
            ASCII 格式的向量 ID
        """
        # 使用 URL 編碼將中文轉換為 ASCII
        encoded_district = urllib.parse.quote(district, safe='')
        return f"taichung_{encoded_district}_{index}"

    def upload_to_pinecone(self, weather_data: List[Dict[str, Any]], 
                          embeddings: List[List[float]]) -> None:
        """
        將天氣資料和嵌入向量上傳到 Pinecone
        
        Args:
            weather_data: 天氣資料列表
            embeddings: 對應的嵌入向量列表
        """
        print("🔄 正在上傳資料到 Pinecone...")
        
        vectors_to_upsert = []
        for i, (data, embedding) in enumerate(zip(weather_data, embeddings)):
            # 使用 ASCII 安全的 vector ID
            vector_id = self._create_safe_vector_id(data['district'], i)
            
            metadata = {
                "district": data["district"],
                "time_range": data["time_range"],
                "avg_temp": data["avg_temp"],
                "min_temp": data["min_temp"],
                "max_temp": data["max_temp"],
                "text_description": data["text_description"]
            }
            
            vectors_to_upsert.append({
                "id": vector_id,
                "values": embedding,
                "metadata": metadata
            })
        
        # 批次上傳向量
        batch_size = 100
        for i in tqdm(range(0, len(vectors_to_upsert), batch_size), desc="上傳向量"):
            batch = vectors_to_upsert[i:i + batch_size]
            self.index.upsert(vectors=batch)
        
        print(f"✅ 成功上傳 {len(vectors_to_upsert)} 個向量到 Pinecone")
    
    def query_weather(self, question: str, top_k: int = 5) -> List[Dict[str, Any]]:
        """
        查詢天氣資料
        
        Args:
            question: 查詢問題
            top_k: 返回的最相關結果數量
            
        Returns:
            最相關的天氣資料列表
        """
        try:
            # 使用 Pinecone Inference API 生成查詢的嵌入向量
            response = self.pc.inference.embed(
                model=self.embedding_model,
                inputs=[question],
                parameters={"input_type": "query"}
            )
            query_embedding = response.data[0].values
            
            # 在 Pinecone 中搜尋
            search_results = self.index.query(
                vector=query_embedding,
                top_k=top_k,
                include_metadata=True
            )
            
            # 格式化結果
            results = []
            for match in search_results.matches:
                result = {
                    "score": match.score,
                    "district": match.metadata["district"],
                    "time_range": match.metadata["time_range"],
                    "avg_temp": match.metadata["avg_temp"],
                    "min_temp": match.metadata["min_temp"],
                    "max_temp": match.metadata["max_temp"],
                    "text_description": match.metadata["text_description"]
                }
                results.append(result)
            
            return results
            
        except Exception as e:
            print(f"❌ 查詢時發生錯誤: {e}")
            return []
    
    def interactive_query(self) -> None:
        """互動式查詢介面"""
        print("\n🎯 天氣查詢系統已啟動！")
        print("您可以詢問關於臺中市天氣的問題，例如：")
        print("- 臺中市中區的溫度如何？")
        print("- 今天臺中最熱的地區是哪裡？")
        print("- 臺中市哪個區域溫差最大？")
        print("輸入 'quit' 退出\n")
        
        while True:
            question = input("🤔 請輸入您的問題: ").strip()
            
            if question.lower() in ['quit', 'exit', '退出', 'q']:
                print("👋 再見！")
                break
            
            if not question:
                continue
            
            print("🔍 正在搜尋相關資料...")
            results = self.query_weather(question)
            
            if not results:
                print("❌ 沒有找到相關資料")
                continue
            
            print(f"\n📊 找到 {len(results)} 個相關結果：\n")
            
            for i, result in enumerate(results[:3], 1):  # 只顯示前3個結果
                print(f"{i}. {result['text_description']}")
                print(f"   相似度: {result['score']:.3f}")
                print()


def main():
    """主程式"""
    # API 金鑰設定
    PINECONE_API_KEY = "pcsk_7PG62n_6tsydkgfVzMfyxhgjP1GCg3DV6r7Qk3mFcT33aQg37ovghyFMNurdfWEA16i831"
    
    # 天氣預報文件路徑
    WEATHER_FILE_PATH = r"E:\python_project\contest\TGIS\code\weather_description.md"
    
    print("🌤️  臺中市天氣預報向量化系統")
    print("=" * 50)
    
    try:
        # 初始化向量化處理器
        vectorizer = WeatherVectorizer(PINECONE_API_KEY)
        
        # 檢查索引狀態
        index_stats = vectorizer.index.describe_index_stats()
        total_vectors = index_stats.total_vector_count
        
        if total_vectors > 0:
            print(f"📊 索引中已有 {total_vectors} 個向量")
            user_input = input("是否要重新處理資料？(y/N): ").strip().lower()
            process_data = user_input in ['y', 'yes', '是']
        else:
            print("📊 索引為空，將開始處理資料")
            process_data = True
        
        if process_data:
            # 提取臺中市天氣資料
            weather_data = vectorizer.extract_taichung_data(WEATHER_FILE_PATH)
            
            if not weather_data:
                print("❌ 未能提取到有效的天氣資料")
                return
            
            # 創建嵌入向量
            texts = [data["text_description"] for data in weather_data]
            embeddings = vectorizer.create_embeddings(texts)
            
            # 上傳到 Pinecone
            vectorizer.upload_to_pinecone(weather_data, embeddings)
        
        # 啟動互動式查詢
        vectorizer.interactive_query()
        
    except Exception as e:
        print(f"❌ 程式執行時發生錯誤: {e}")


if __name__ == "__main__":
    main()



🌤️  臺中市天氣預報向量化系統
✅ 已初始化 Pinecone 索引: weather-txt-index
✅ 已初始化 Pinecone 嵌入模型: llama-text-embed-v2
📊 索引為空，將開始處理資料
✅ 成功提取臺中市 435 筆天氣資料
🔄 正在生成文本嵌入向量...


生成嵌入向量: 100%|██████████| 14/14 [00:09<00:00,  1.50it/s]


✅ 成功生成 435 個嵌入向量
🔄 正在上傳資料到 Pinecone...


上傳向量: 100%|██████████| 5/5 [00:06<00:00,  1.33s/it]


✅ 成功上傳 435 個向量到 Pinecone

🎯 天氣查詢系統已啟動！
您可以詢問關於臺中市天氣的問題，例如：
- 臺中市中區的溫度如何？
- 今天臺中最熱的地區是哪裡？
- 臺中市哪個區域溫差最大？
輸入 'quit' 退出



: 